<a href="https://colab.research.google.com/github/TheAllBlueChevy/UST-Deep-Learning-2026/blob/main/HWK5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **HWK5**

# 0) Importing Libraries

In [1]:
import time
import numpy as np
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Dropout, Conv1D, Flatten
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

np.random.seed(42)
tf.random.set_seed(42)

# 1) Data Set Up

In [2]:
samples = 5000
length = 400
channels = 4
classes = 3

X = np.random.rand(samples, length, channels).astype("float32")
y = np.random.randint(0, classes, size=(samples,))

In [3]:
X_train, X_val, y_train, y_val = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)


In [4]:
print(y_train.shape)
print(y_train[:5])

(4000,)
[1 0 0 2 0]


# 2) Model Builder

In [5]:
def cnn_model():
    model = Sequential([
        Conv1D(
            filters=50,
            kernel_size=9,
            activation="relu",
            input_shape=(400, 4)
        ),
        Flatten(),
        Dense(100, activation='relu'),
        Dense(3, activation='softmax')
    ])

    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

In [6]:
model = cnn_model()
model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 392, 50)        │         1,850 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 19600)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 100)            │     1,960,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │           303 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,962,253 (7.49 MB)

 Trainable params: 1,962,253 (7.49 MB)

 Non-trainable params: 0 (0.00 B)

In [7]:
early_stop = EarlyStopping(
    monitor="val_loss",
    patience=3,
    restore_best_weights=True
)

# 3) Training on CPU/GPU

In [8]:
tf.keras.backend.clear_session()
with tf.device("/CPU:0"):
    cpu_model = cnn_model()

    start_time = time.time()
    cpu_model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=50,
        batch_size=32,
        callbacks=[early_stop],
        verbose=1
    )
    cpu_time = time.time() - start_time

Epoch 1/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 8s 51ms/step - accuracy: 0.3513 - loss: 1.2581 - val_accuracy: 0.3280 - val_loss: 1.1016
Epoch 2/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 4s 30ms/step - accuracy: 0.4484 - loss: 1.0560 - val_accuracy: 0.3210 - val_loss: 1.1226
Epoch 3/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 4s 32ms/step - accuracy: 0.6094 - loss: 0.9097 - val_accuracy: 0.3120 - val_loss: 1.1631
Epoch 4/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 4s 26ms/step - accuracy: 0.7868 - loss: 0.6408 - val_accuracy: 0.3120 - val_loss: 1.2220


In [9]:
tf.keras.backend.clear_session()
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [10]:
gpu_time = None

if tf.config.list_physical_devices("GPU"):
    with tf.device("/GPU:0"):
        gpu_model = cnn_model()

        start_time = time.time()
        gpu_model.fit(
            X_train, y_train,
            validation_data=(X_val, y_val),
            epochs=50,
            batch_size=32,
            callbacks=[early_stop],
            verbose=1
        )
        gpu_time = time.time() - start_time
else:
    print("No GPU detected.")


Epoch 1/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.3335 - loss: 1.1977 - val_accuracy: 0.3320 - val_loss: 1.0992
Epoch 2/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4603 - loss: 1.0723 - val_accuracy: 0.3290 - val_loss: 1.1093
Epoch 3/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5783 - loss: 0.9594 - val_accuracy: 0.3180 - val_loss: 1.1350
Epoch 4/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7494 - loss: 0.7051 - val_accuracy: 0.3310 - val_loss: 1.2784


# 4) Results

In [11]:
print("\nTraining Time Comparison")
print(f"CPU Time: {cpu_time:.2f} seconds")

if gpu_time is not None:
    print(f"GPU Time: {gpu_time:.2f} seconds")
    print(f"Faster By: {cpu_time / gpu_time:.2f}×")



Training Time Comparison
CPU Time: 20.58 seconds
GPU Time: 5.71 seconds
Faster By: 3.60×
